# Using Selenium Automation chooses best selling products on Tiki

In [3]:
# Import package
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import pandas as pd
import re
from time import sleep
import os
import csv

# Input values need to find
search_query = input('What products are you looking for? ')
pages = int(input('How many pages? '))
sold = int(input('Request? How many products are sold? '))

# Access Tiki's Website
url = 'https://tiki.vn'
driver = webdriver.Chrome(service=Service(r"E:\OneDrive\CNTT\test\chromedriver")) 
driver.get(url)
sleep(3)

# Send keyword
search_field = driver.find_element(by=By.XPATH, value='//*[@id="__next"]/div[1]/header/div[1]/div/div[1]/div[2]/div/input') #tìm thành phần Search
search_field.send_keys(search_query)
sleep(1)
search_field.send_keys(Keys.RETURN) #nhấn nút Enter
print('Complete product search')
sleep(3)

# Choose 5 star review
driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
sleep(2)
driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/main/div[2]/div/div[1]/div[4]/div/a[1]').click()
print('Complete the 5 star review filter')
sleep(3)

all_list_info = []
all_list_link = []
for page in range(1,pages+1):
    print(f'Page {page}')
    # Getting product information
    list_info = []
    info = driver.find_elements(By.CLASS_NAME,"info")
    for i in info:
        list_info.append(i.text)
    sleep(2)

    # Getting URL products
    list_link = [] 
    link = driver.find_elements(By.CLASS_NAME,"product-item")
    for i in link:
        list_link.append(i.get_attribute("href"))
    print('Complete getting information product')
    all_list_info = all_list_info + list_info
    all_list_link = all_list_link + list_link
    sleep(2)
    
    # Next page
    pagination = driver.find_element(By.CLASS_NAME,'Pagination__Root-sc-cyke21-0.gNgpAR')
    driver.execute_script("arguments[0].scrollIntoView();", pagination)
    next_bottom = pagination.find_element(By.CLASS_NAME,'tikicon.icon-arrow-back')
    sleep(3)
    next_bottom.click()    
    sleep(4)
driver.close()    

# String Manipulation
list_new = []
for i in all_list_info:
    i = i.replace('GIAO SIÊU TỐC 2H\n','').replace('GIAO SÁNG MAI\n','').replace('GIAO TRONG NGÀY\n','').replace('\nTrả góp','').replace('\nQuà tặng kèm','').replace('\nHàng quốc tế','').replace('CHỌN GIỜ GIAO LẮP\n','').replace('Ad\n','')
    list_new.append(i)   
list_final = []
for i in list_new:
    remove_promotion = re.sub('\\n-(\d+%|)', '', i)
    list_final.append(remove_promotion)
list_split = []
for i in list_final:
        list_split.append(i.split('\n'))

# Create a DataFrame and processing data
df = pd.DataFrame(columns=['Name','Sold','Price'], data = list_split)
print(f'Found {len(df)} products')
df['Link'] = all_list_link
df = df.dropna()
df_new = df.copy()
df_new['Sold'] = df_new['Sold'].str.replace('Đã bán ', '', regex=True)
df_new['Sold'] = df_new['Sold'].str.replace('+','',regex=True).astype(int)

df_filter = df.loc[df_new['Sold']>sold]
print(f'There are {len(df_filter)} products that meet your requirements')
df_filter.to_csv(r'E:\OneDrive\CNTT\test\tiki.csv', index=False)
os.startfile(r'E:\OneDrive\CNTT\test\tiki.csv')
df_filter

What products are you looking for? áo mưa
How many pages? 3
Request? How many products are sold? 200
Complete product search
Complete the 5 star review filter
Page 1
Complete getting information product
Page 2
Complete getting information product
Page 3
Complete getting information product
Found 148 products
There are 37 products that meet your requirements


,Name,Sold,Price,Link
1,Áo Mưa Bộ Vải Dù Cao Cấp ( 2 size ),Đã bán 524,98.000 ₫,https://tiki.vn/ao-mua-bo-vai-du-cao-cap-2-siz...
2,Áo mưa dây kéo vải dù tổ ong chống thấm ( nhiề...,Đã bán 1000+,119.000 ₫,https://tiki.vn/ao-mua-day-keo-vai-du-to-ong-c...
3,Bộ Quần Áo Mưa Vải Dù Từ 50-80kg 2 Lớp Khóa Kh...,Đã bán 669,95.000 ₫,https://tiki.vn/bo-quan-ao-mua-vai-du-tu-50-80...
5,"Áo Mưa Bộ Vải Dù, Bộ Quần Áo Mưa Có Phản Quang...",Đã bán 337,142.000 ₫,https://tiki.vn/ao-mua-bo-vai-du-bo-quan-ao-mu...
8,"Áo Mưa Cánh Dơi Có Kiếng Đèn, Áo Mưa Kiếng Đèn...",Đã bán 311,125.000 ₫,https://tiki.vn/ao-mua-canh-doi-co-kieng-den-a...
10,"Áo Mưa Bít Dù Trơn, Áo Mưa Kín Người Vải Dù Si...",Đã bán 506,115.000 ₫,https://tiki.vn/ao-mua-bit-du-tron-ao-mua-kin-...
11,Áo Mưa Vải Dù Cánh Dơi Dài 1m4 Chống Thấm,Đã bán 417,72.000 ₫,https://tiki.vn/ao-mua-vai-du-canh-doi-dai-1m4...
14,Áo Mưa Bít Không Xẻ Tà Vải Dù Chống Thấm Dài 1...,Đã bán 384,77.000 ₫,https://tiki.vn/ao-mua-bit-khong-xe-ta-vai-du-...
15,ÁO MƯA BIT NÂNG CAO,Đã bán 210,82.900 ₫,https://tiki.vn/ao-mua-bit-nang-cao-p16107196....
19,Áo mưa bộ 2 lớp kiêm áo gió Lotusrain (Hàng ch...,Đã bán 219,305.000 ₫,https://tiki.vn/ao-mua-bo-2-lop-kiem-ao-gio-lo...
